# Chapter 7: SLAM (Simultaneous Localization and Mapping)

## Principles of Indoor Positioning and Indoor Navigation

---

### 📚 Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** the SLAM problem (chicken-and-egg: localization ↔ mapping)
2. **Implement** scan matching algorithms (ICP) for relative pose estimation
3. **Build** pose graphs with odometry and loop closure factors
4. **Optimize** pose graphs to correct accumulated drift
5. **Analyze** how loop closures dramatically improve trajectory estimation

### 📖 Book Reference

This notebook covers **Chapter 7: SLAM** with:
- **Section 7.2**: Scan Matching (ICP, NDT)
- **Section 7.3**: Pose Graph Optimization
- **Eq. (7.10)-(7.11)**: ICP point-to-point alignment
- **Eq. (7.40)-(7.46)**: Camera models for visual SLAM

**Key Insight**: SLAM can reduce drift by 90%+ through loop closure detection!

---


## 🚀 Setup (Google Colab)

**Set the `GITHUB_REPO` variable below to your repository URL, then run the setup cell.**

Example: `GITHUB_REPO = "https://github.com/YOUR_USERNAME/IPIN-Examples.git"`


In [ ]:
# ========================================
# IPIN Book Examples - Chapter 7: SLAM
# ========================================

import os
import sys

# ============ CONFIGURATION ============
GITHUB_REPO = None  # Set your repo URL, e.g., "https://github.com/username/IPIN-Examples.git"
# =======================================

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    if os.path.exists('/content/IPIN-Examples/core'):
        os.chdir('/content/IPIN-Examples')
        print("✅ Repository already available.")
    elif GITHUB_REPO:
        print(f"📥 Cloning from {GITHUB_REPO}...")
        get_ipython().system(f'git clone {GITHUB_REPO}')
        os.chdir('/content/IPIN-Examples')
        get_ipython().system('pip install -e . -q')
        print("✅ Setup from GitHub complete!")
    else:
        print("❌ ERROR: GITHUB_REPO not set!")
        print("Please set GITHUB_REPO = 'https://github.com/YOUR_USERNAME/IPIN-Examples.git'")
        raise ValueError("GITHUB_REPO not configured.")
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
    print(f"📂 Working directory: {os.getcwd()}")

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

# Import SLAM modules
from core.slam import (
    se2_apply,
    se2_compose,
    se2_relative,
    icp_point_to_point,
    create_pose_graph,
)

print("\n" + "="*60)
print("✅ Setup complete! SLAM modules loaded.")
print("="*60)


---

# Part 1: SLAM Overview

## 1.1 The SLAM Problem

**SLAM** solves the chicken-and-egg problem:

| Need | Problem |
|------|---------|
| **Localization** requires a map | But we don't have a map! |
| **Mapping** requires knowing location | But we don't know our location! |
| **SLAM** | Solves both simultaneously |

## 1.2 SLAM Pipeline

```
┌─────────────┐    ┌──────────────┐    ┌─────────────┐    ┌───────────┐
│   Sensors   │ →  │ Scan Matching│ →  │ Pose Graph  │ →  │ Optimized │
│ (LiDAR/Cam) │    │   (ICP/NDT)  │    │ Construction│    │ Trajectory│
└─────────────┘    └──────────────┘    └─────────────┘    └───────────┘
                                              ↓
                                       ┌─────────────┐
                                       │Loop Closure │
                                       │  Detection  │
                                       └─────────────┘
```

## 1.3 Key Insight

**Loop closures** are the magic ingredient:
- When robot revisits a known location, it "closes the loop"
- This constraint corrects accumulated drift throughout the trajectory
- Without loop closures, SLAM is just odometry (drifts unbounded)


---

# Part 2: Simulating a SLAM Scenario

## 2.1 Scenario: Square Loop Trajectory

We'll simulate a robot traversing a square loop (returns to start), demonstrating:
1. Odometry drift accumulates over the trajectory
2. Loop closure detects "we're back at the start!"
3. Optimization distributes the error correction


In [ ]:
# Generate ground truth trajectory (square loop)
print("="*70)
print("Example 1: Pose Graph SLAM (Square Loop Trajectory)")
print("="*70)

np.random.seed(42)

# Square loop parameters
side_length = 10.0  # meters
n_poses_per_side = 5
n_poses = 4 * n_poses_per_side + 1  # Return to start

# Generate ground truth poses [x, y, theta]
true_poses = []
step = side_length / n_poses_per_side

# Side 1: East
for i in range(n_poses_per_side):
    true_poses.append([i * step, 0.0, 0.0])

# Side 2: North
for i in range(n_poses_per_side):
    true_poses.append([side_length, i * step, np.pi/2])

# Side 3: West
for i in range(n_poses_per_side):
    true_poses.append([side_length - i * step, side_length, np.pi])

# Side 4: South
for i in range(n_poses_per_side):
    true_poses.append([0.0, side_length - i * step, -np.pi/2])

# Return to start (loop closure!)
true_poses.append([0.0, 0.0, 0.0])

true_poses = np.array(true_poses)
n_poses = len(true_poses)

print(f"\n📍 Ground Truth Trajectory:")
print(f"  Total poses: {n_poses}")
print(f"  Path: Square loop ({side_length}m × {side_length}m)")
print(f"  Total distance: {4 * side_length:.1f} m")


In [ ]:
# Simulate odometry with drift
odom_noise_trans = 0.1   # 10 cm translational noise per step
odom_noise_rot = 0.05    # ~3 deg rotational noise per step

# Compute noisy odometry
odom_poses = [true_poses[0].copy()]

for i in range(1, n_poses):
    # True relative pose
    rel_true = se2_relative(true_poses[i-1], true_poses[i])
    
    # Add noise to relative pose
    rel_noisy = rel_true.copy()
    rel_noisy[0] += np.random.randn() * odom_noise_trans
    rel_noisy[1] += np.random.randn() * odom_noise_trans
    rel_noisy[2] += np.random.randn() * odom_noise_rot
    
    # Compose with previous pose
    new_pose = se2_compose(odom_poses[-1], rel_noisy)
    odom_poses.append(new_pose)

odom_poses = np.array(odom_poses)

# Compute final drift (should be non-zero since we have noise)
final_drift = np.linalg.norm(odom_poses[-1, :2] - true_poses[-1, :2])
print(f"\n🚨 Odometry Drift (without SLAM):")
print(f"  Final position error: {final_drift:.3f} m")
print(f"  (Robot thinks it's at [{odom_poses[-1, 0]:.2f}, {odom_poses[-1, 1]:.2f}]")
print(f"   but actually at [{true_poses[-1, 0]:.2f}, {true_poses[-1, 1]:.2f}])")


---

# Part 3: Pose Graph Construction

## 3.1 Pose Graph Structure

A **pose graph** represents the SLAM problem as a graph:
- **Nodes**: Robot poses (variables to optimize)
- **Edges**: Constraints between poses
  - **Odometry edges**: Connect consecutive poses
  - **Loop closure edges**: Connect non-consecutive poses that observe the same place

```
pose_0 ──odom── pose_1 ──odom── pose_2 ── ... ── pose_N
   │                                                 │
   └──────────────── loop closure ───────────────────┘
```


In [ ]:
# Build pose graph
print("="*70)
print("Building Pose Graph")
print("="*70)

# Create odometry measurements (edges between consecutive poses)
odometry_measurements = []
for i in range(n_poses - 1):
    # Use noisy relative pose as measurement
    rel_pose = se2_relative(odom_poses[i], odom_poses[i+1])
    odometry_measurements.append((i, i+1, rel_pose))

print(f"\n📊 Odometry edges: {len(odometry_measurements)}")

# Create loop closure: pose_N-1 should be same as pose_0 (back at start!)
# The loop closure measurement says "pose N-1 is at same location as pose 0"
loop_closure_rel = se2_relative(true_poses[-1], true_poses[0])  # Should be identity!
loop_closures = [(n_poses-1, 0, loop_closure_rel)]

print(f"🔄 Loop closure: pose_{n_poses-1} ↔ pose_0")
print(f"   (Robot detected it returned to the starting position!)")

# Information matrices (inverse covariance = confidence)
odom_info = np.diag([100.0, 100.0, 1000.0])    # Moderate confidence
loop_info = np.diag([400.0, 400.0, 4000.0])    # High confidence (same place!)

# Build pose graph
graph = create_pose_graph(
    poses=odom_poses.tolist(),
    odometry_measurements=odometry_measurements,
    loop_closures=loop_closures,
    odometry_information=odom_info,
    loop_information=loop_info,
)

print(f"\n📈 Pose Graph Statistics:")
print(f"  Variables (poses): {len(graph.variables)}")
print(f"  Factors (constraints): {len(graph.factors)}")


In [ ]:
# Optimize pose graph
print("="*70)
print("Optimizing Pose Graph")
print("="*70)

# Compute error before optimization
initial_error = graph.compute_error()
print(f"\n📊 Initial error: {initial_error:.4f}")

# Optimize! (Note: parameter is 'tol', not 'tolerance')
optimized_vars, error_history = graph.optimize(max_iterations=50, tol=1e-6)

# Convert optimized variables dict to array (variables are indexed 0, 1, 2, ...)
optimized_poses = np.array([optimized_vars[i] for i in range(len(optimized_vars))])

final_error = graph.compute_error()
print(f"📊 Final error: {final_error:.4f}")
print(f"📉 Error reduction: {100*(1 - final_error/initial_error):.2f}%")
print(f"⏱️ Iterations: {len(error_history) - 1}")

# Compute RMSE before and after
odom_rmse = np.sqrt(np.mean(np.sum((odom_poses[:, :2] - true_poses[:, :2])**2, axis=1)))
slam_rmse = np.sqrt(np.mean(np.sum((optimized_poses[:, :2] - true_poses[:, :2])**2, axis=1)))

print(f"\n🎯 Position RMSE:")
print(f"  Odometry only: {odom_rmse:.4f} m")
print(f"  After SLAM:    {slam_rmse:.4f} m")
print(f"  Improvement:   {100*(1 - slam_rmse/odom_rmse):.1f}%")


In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Trajectory comparison
ax1 = axes[0]
ax1.plot(true_poses[:, 0], true_poses[:, 1], 'g-', linewidth=2, label='Ground Truth', marker='o', markersize=5)
ax1.plot(odom_poses[:, 0], odom_poses[:, 1], 'r--', linewidth=2, label='Odometry (drifted)', marker='x', markersize=5)
ax1.plot(optimized_poses[:, 0], optimized_poses[:, 1], 'b-', linewidth=2, label='SLAM Optimized', marker='s', markersize=5)

# Mark start/end
ax1.plot(true_poses[0, 0], true_poses[0, 1], 'ko', markersize=12, label='Start', zorder=10)

# Draw loop closure
ax1.annotate('', xy=(odom_poses[-1, 0], odom_poses[-1, 1]), 
             xytext=(odom_poses[0, 0], odom_poses[0, 1]),
             arrowprops=dict(arrowstyle='->', color='purple', lw=2, ls='--'))
ax1.text(0.5, -1.5, 'Loop Closure!', fontsize=10, color='purple', ha='center')

ax1.set_xlabel('X (m)', fontsize=12)
ax1.set_ylabel('Y (m)', fontsize=12)
ax1.set_title('Pose Graph SLAM: Trajectory Comparison', fontsize=14, fontweight='bold')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Plot 2: Error over trajectory
ax2 = axes[1]
odom_errors = np.linalg.norm(odom_poses[:, :2] - true_poses[:, :2], axis=1)
slam_errors = np.linalg.norm(optimized_poses[:, :2] - true_poses[:, :2], axis=1)

pose_indices = np.arange(len(true_poses))
ax2.plot(pose_indices, odom_errors, 'r-', linewidth=2, label='Odometry Error')
ax2.plot(pose_indices, slam_errors, 'b-', linewidth=2, label='SLAM Error')
ax2.axvline(x=len(pose_indices)-1, color='purple', linestyle='--', label='Loop Closure')
ax2.fill_between(pose_indices, odom_errors, slam_errors, alpha=0.3, color='green', label='SLAM Improvement')

ax2.set_xlabel('Pose Index', fontsize=12)
ax2.set_ylabel('Position Error (m)', fontsize=12)
ax2.set_title('Error Comparison Over Trajectory', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: Loop closure distributes error correction across ENTIRE trajectory!")


---

# Part 4: Scan Matching (ICP)

## 4.1 ICP Algorithm (Eqs. 7.10-7.11)

**Iterative Closest Point (ICP)** aligns two point clouds:

1. **Find correspondences**: For each point in source, find nearest in target
2. **Compute transform**: Find rotation R and translation t that minimizes:
   $$\min_{R,t} \sum_i \| R \mathbf{p}_i + t - \mathbf{q}_{c(i)} \|^2$$
3. **Apply transform** to source points
4. **Repeat** until convergence

ICP is used for:
- **Loop closure detection**: Match current scan to previous scans
- **Relative pose estimation**: Determine robot motion between scans


In [ ]:
# Example 2: ICP Scan Matching
print("="*70)
print("Example 2: ICP Scan Matching")
print("="*70)

np.random.seed(123)

# Generate a simple 2D point cloud (L-shape)
target = np.array([
    [0, 0], [1, 0], [2, 0], [3, 0], [4, 0],
    [0, 1], [0, 2], [0, 3], [0, 4],
], dtype=float)

# Transform source with known rotation and translation
true_angle = np.deg2rad(15)  # 15 degree rotation
true_trans = np.array([0.5, 0.3])

R_true = np.array([
    [np.cos(true_angle), -np.sin(true_angle)],
    [np.sin(true_angle), np.cos(true_angle)]
])
source = (R_true @ target.T).T + true_trans

# Add noise to source
source += np.random.randn(*source.shape) * 0.05

print(f"\n📊 ICP Test:")
print(f"  Target points: {len(target)}")
print(f"  Source points: {len(source)}")
print(f"  True rotation: {np.rad2deg(true_angle):.1f}°")
print(f"  True translation: [{true_trans[0]:.2f}, {true_trans[1]:.2f}]")

# Run ICP
R_est, t_est, info_icp = icp_point_to_point(source, target, max_iterations=50)

# Extract estimated angle
est_angle = np.arctan2(R_est[1, 0], R_est[0, 0])

print(f"\n🎯 ICP Results:")
print(f"  Estimated rotation: {np.rad2deg(est_angle):.1f}°")
print(f"  Estimated translation: [{t_est[0]:.2f}, {t_est[1]:.2f}]")
print(f"  Rotation error: {np.abs(np.rad2deg(est_angle - true_angle)):.2f}°")
print(f"  Translation error: {np.linalg.norm(t_est - true_trans):.4f} m")
print(f"  Iterations: {info_icp['iterations']}")


In [ ]:
# Visualize ICP result
aligned_source = (R_est @ source.T).T + t_est

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Before alignment
ax1 = axes[0]
ax1.scatter(target[:, 0], target[:, 1], c='blue', s=100, marker='o', label='Target')
ax1.scatter(source[:, 0], source[:, 1], c='red', s=100, marker='x', label='Source (before)')
ax1.set_title('Before ICP Alignment', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')
ax1.set_xlabel('X (m)')
ax1.set_ylabel('Y (m)')

# After alignment
ax2 = axes[1]
ax2.scatter(target[:, 0], target[:, 1], c='blue', s=100, marker='o', label='Target')
ax2.scatter(aligned_source[:, 0], aligned_source[:, 1], c='green', s=100, marker='s', label='Source (after ICP)')
ax2.set_title('After ICP Alignment', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')
ax2.set_xlabel('X (m)')
ax2.set_ylabel('Y (m)')

plt.tight_layout()
plt.show()

print("\n💡 ICP finds the transform that aligns point clouds - used for loop closure detection!")


---

# Summary

## Key Takeaways

### 1. SLAM Components

| Component | Purpose | Method |
|-----------|---------|--------|
| **Scan Matching** | Find relative pose between scans | ICP, NDT |
| **Pose Graph** | Represent trajectory + constraints | Graph structure |
| **Loop Closure** | Detect revisited locations | Scan matching similarity |
| **Optimization** | Minimize constraint errors | Gauss-Newton, LM |

### 2. Performance Impact

| Scenario | RMSE | Notes |
|----------|------|-------|
| **Odometry Only** | ~0.5-2 m | Unbounded drift |
| **SLAM + Loop Closure** | ~0.05 m | 90-95% improvement! |

### 3. Critical Insights

1. **Loop closures are essential** - without them, SLAM is just odometry
2. **More loop closures = better** - each one adds a constraint
3. **ICP provides relative poses** - used for both odometry and loop closure detection
4. **Optimization distributes error** - correction spreads across entire trajectory

---

## SLAM Variants

| Type | Sensors | Applications |
|------|---------|--------------|
| **LiDAR SLAM** | 2D/3D LiDAR | Robotics, mapping |
| **Visual SLAM** | Camera | AR/VR, drones |
| **Visual-Inertial** | Camera + IMU | Mobile devices |

---

## Exercises

1. **More loop closures**: What if you detect 3 loop closures instead of 1?
2. **Higher drift**: Increase odometry noise - how much can SLAM compensate?
3. **Different trajectories**: Try a figure-8 path with multiple loop closures

---

**Next Steps:** Chapter 8 (Sensor Fusion) to combine multiple positioning sources!
